In [ ]:
import petl as etl
import psycopg2

# Database connection parameters
db_params = {
    'dbname': 'your_database',
    'user': 'your_username',
    'password': 'your_password',
    'host': 'localhost',  # or your database host
    'port': '5432'        # default PostgreSQL port
}

# Step 1: Extract data from CSV
# Replace 'input.csv' with your actual CSV file path
table = etl.fromcsv('input.csv')

# Step 2: Transform data
# Example transformations:
# 1. Rename columns
table = etl.rename(table, {'old_column_name': 'new_column_name'})

# 2. Filter rows (e.g., where a column value is greater than a threshold)
table = etl.cutout(table, 'unwanted_column')  # Remove unwanted column
table = etl.select(table, '{column_name} > 100')  # Filter rows based on a condition

# 3. Add a new calculated column
table = etl.addfield(table, 'new_calculated_column', lambda rec: rec['existing_column'] * 2)

# Step 3: Load data into PostgreSQL
# Create a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Create a table in PostgreSQL (if it doesn't exist)
cur.execute("""
CREATE TABLE IF NOT EXISTS your_table (
    new_column_name VARCHAR(255),
    new_calculated_column INTEGER
);
""")
conn.commit()

# Load data into PostgreSQL
etl.todb(table, conn, 'your_table', ifexists='append')

# Close the database connection
cur.close()
conn.close()

print("ETL process completed successfully.")
